# LLM API로 간단한 챗봇 만들기

## 1. OpenAI API

### 필요한 모듈
pip install openai  
pip install python-dotenv  
pip install ipykernel  

Installing collected packages: typing-extensions, sniffio, jiter, idna, h11, distro, colorama, certifi, annotated-types, typing-inspection, tqdm, pydantic-core, httpcore, anyio, pydantic, httpx, openai

### Single Turn Chatbot

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    temperature = 0.5, # 1에 가까울수록 창의적 답변
    messages = [
        {"role":"system", "content":"너는 사용자를 도와주는 상담사야."},
        {"role":"user", "content":"한국의 수도는 어디야?"},
    ]
    )
print(response)

ChatCompletion(id='chatcmpl-CCkjSsBrHrmdKOUnaKfqmtXSedK6x', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='한국의 수도는 서울입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757155610, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_cbf1785567', usage=CompletionUsage(completion_tokens=7, prompt_tokens=29, total_tokens=36, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [3]:
print("AI: " + response.choices[0].message.content)

AI: 한국의 수도는 서울입니다.


In [ ]:
client = OpenAI()

while True:
    user_input = input("사용자: ")
    if user_input == "exit":
        break
    response = client.chat.completions.create(
    model="gpt-4o",
    temperature = 0.5,
    messages = [
        {"role":"system", "content":"너는 사용자를 도와주는 친절한 조력자야."},
        {"role":"user", "content":user_input},
    ]
    )
    print("AI: " + response.choices[0].message.content)

### Multi-turn Chatbot

In [ ]:
client = OpenAI()

def get_ai_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature = 0.5,
        messages = messages,
    )
    return response.choices[0].message.content

messages = [
    {"role":"system", "content":"너는 사용자를 도와주는 친절한 조력자야."}
]

while True:
    user_input = input("사용자: ")
    if user_input == "exit":
        break
    messages.append({"role": "user", "content": user_input})
    ai_msg = get_ai_response(messages)
    messages.append({"role": "assistant", "content": ai_msg})
    print("AI: "+ ai_msg)




AI: 안녕하세요, 이용환님! 만나서 반갑습니다. 어떻게 도와드릴까요?
AI: 일본의 수도는 도쿄입니다. 도쿄는 일본의 정치, 경제, 문화의 중심지로 잘 알려져 있습니다. 추가로 궁금한 점이 있으면 언제든지 물어보세요!
AI: 당신은 자신을 이용환이라고 소개하셨어요. 다른 궁금한 점이 있으면 말씀해 주세요!


## 2. Anthropic API

pip install anthropic  

models: https://docs.anthropic.com/en/docs/resources/model-deprecations

In [ ]:
import anthropic

client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=0.5,  # 1에 가까울수록 창의적 답변
    system="너는 사용자를 도와주는 친절한 조력자야.",
    messages=[
        {"role": "user", "content": "한국의 수도는 어디야?"}
    ]
)

print(response)

Message(id='msg_012DFPiQ9m8E53NYMs5WdCz3', content=[TextBlock(citations=None, text='한국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로, 약 1,000만 명의 인구가 거주하고 있는 대도시입니다.', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=38, output_tokens=74, server_tool_use=None, service_tier='standard'))


In [7]:
print('AI :' + response.content[0].text)

AI :한국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로, 약 1,000만 명의 인구가 거주하고 있는 대도시입니다.


# Function Calling

In [ ]:
# 테스트 질문: 지금 몇 시야?

client = OpenAI()

def get_ai_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature = 0.5,
        messages = messages,
    )
    return response.choices[0].message.content

messages = [
    {"role":"system", "content":"너는 사용자를 도와주는 친절한 조력자야."}
]

while True:
    user_input = input("사용자: ")
    if user_input == "exit":
        break
    messages.append({"role": "user", "content": user_input})
    ai_msg = get_ai_response(messages)
    messages.append({"role": "assistant", "content": ai_msg})
    print("AI: "+ ai_msg)

AI: 죄송하지만, 현재 시간을 확인해 드릴 수는 없습니다. 사용 중인 기기나 시계를 통해 현재 시간을 확인해 보세요.


pip install pytz

In [ ]:
from datetime import datetime
import pytz

def get_current_time(timezone: str = 'Asia/Seoul') -> str:
    tz = pytz.timezone(timezone)
    current_time = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    result = f'{current_time} {timezone}'
    print(result)
    return result

In [18]:
current_time = get_current_time('Asia/Seoul')
# current_time = get_current_time('America/New_York')
# current_time = get_current_time('유럽/런던') # Error

2025-09-06 22:35:46 Asia/Seoul


In [20]:
tools = [
    {
        "type":"function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 현재 날짜와 시간을 타임존과 함께 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환받고자 하는 타임존을 입력하세요. (예: Asian/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    }      

]

In [ ]:
# 테스트 질문: 지금 몇 시야? 웹 검색 가능해?

import json

client = OpenAI()

def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages = messages,
        tools=tools,
    )
    return response

messages = [
    {"role":"system", "content":"너는 사용자를 도와주는 친절한 조력자야."}
]


while True:
    user_input = input("사용자: ")
    if user_input == "exit":
        break
    messages.append({"role": "user", "content": user_input})
    response = get_ai_response(messages, tools=tools)
    ai_msg = response.choices[0].message
    print(ai_msg)

    tool_calls = ai_msg.tool_calls

    # 하나의 지역 시간을 묻는 경우
    if tool_calls:
        tool_name = tool_calls[0].function.name
        tool_call_id = tool_calls[0].id

        arguments = json.loads(tool_calls[0].function.arguments) # str -> dict

        if tool_name == "get_current_time":
            messages.append({
                "role": "function",
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_current_time(timezone=arguments['timezone']),
            })
        response = get_ai_response(messages, tools=tools)
        ai_msg = response.choices[0].message

    # 여러 지역 시간을 묻는 경우
    # if tool_calls:
    #     for tool_call in tool_calls:
    #         tool_name = tool_call.function.name
    #         tool_call_id = tool_call.id

    #         arguments = json.loads(tool_call.function.arguments)

    #         if tool_name == "get_current_time":
    #             messages.append({
    #                 "role": "function",
    #                 "tool_call_id": tool_call_id,
    #                 "name": tool_name,
    #                 "content": get_current_time(timezone=arguments['timezone']),
    #             })    
    #     messages.append({"role":"system", "content":"이제 주어진 결과를 참고해서 답변해."})
    #     response = get_ai_response(messages, tools=tools)
    #     ai_msg = response.choices[0].message

    messages.append(ai_msg)
    print("AI: " + ai_msg.content)


ChatCompletionMessage(content='안녕하세요, 이용환님! 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
AI: 안녕하세요, 이용환님! 어떻게 도와드릴까요?
ChatCompletionMessage(content='어느 타임존의 현재 시간을 알고 싶으신가요? 예를 들어, 서울 시간 또는 다른 지역의 시간을 말씀해 주세요.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
AI: 어느 타임존의 현재 시간을 알고 싶으신가요? 예를 들어, 서울 시간 또는 다른 지역의 시간을 말씀해 주세요.
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_HJfw8YNPGAoeMbGg8h1oyytL', function=Function(arguments='{"timezone":"Asia/Seoul"}', name='get_current_time'), type='function')])
2025-09-06 22:49:21 Asia/Seoul
AI: 서울의 현재 시간은 2025년 9월 6일 오후 10시 49분입니다. 다른 도움이 필요하시면 말씀해 주세요!
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionTool

pip install yfinance

Installing collected packages: peewee, multitasking, websockets, urllib3, tzdata, soupsieve, pycparser, protobuf, numpy, frozendict, charset_normalizer, requests, pandas, cffi, beautifulsoup4, curl_cffi, yfinance  

pip install tabulate

In [23]:
import yfinance as yf

def get_yf_stock_info(ticker: str):
    stock = yf.Ticker(ticker)
    info = stock.info
    print(info)
    return str(info)

def get_yf_stock_history(ticker: str, period: str):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    history_md = history.to_markdown()
    print(history_md)
    return history_md

def get_yf_stock_recommendations(ticker: str):
    stock = yf.Ticker(ticker)
    recommendations = stock.recommendations
    recommendations_md = recommendations.to_markdown()
    print(recommendations_md)
    return recommendations_md

tools = [
    {
        "type":"function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 현재 날짜와 시간을 타임존과 함께 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환받고자 하는 타임존을 입력하세요. (예: Asian/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },

    {
        "type":"function",
        "function": {
            "name": "get_yf_stock_info",
            "description": "해당 종목의 Yahoo Finance 제공 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 에서 정보를 조회할 종목의 티커를 입력하세요. (예: TSLA)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
    {
        "type":"function",
        "function": {
            "name": "get_yf_stock_history",
            "description": "해당 종목의 Yahoo Finance 제공 주가 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 에서 주가 정보를 조회할 종목의 티커를 입력하세요. (예: TSLA)',
                    },
                    'period': {
                        'type': 'string',
                        'description': '주가 정보를 알고 싶은 기간을 입력하세요. (예: 1d, 5d, 1mo, 1y, 5y)',
                    },
                },
                "required": ['ticker', 'period'],
            },
        }
    },
    {
        "type":"function",
        "function": {
            "name": "get_yf_stock_recommendations",
            "description": "해당 종목에 대한 Yahoo Finance 제공 추천 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 에서 추천 정보를 조회할 종목의 티커를 입력하세요. (예: TSLA)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },      

]

In [ ]:
get_yf_stock_info('NVDA')

{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation, a computing infrastructure company, provides graphics and compute and networking solutions in the United States, Singapore, Taiwan, China, Hong Kong, and internationally. The Compute & Networking segment includes its Data Centre accelerated computing platforms and artificial intelligence solutions and software; networking; automotive platforms and autonomous and electric vehicle solutions; Jetson for robotics and other embedded platforms; and DGX Cloud computing services. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infra

In [27]:
get_yf_stock_history('TSLA', '5d')

| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|
| 2025-08-29 00:00:00-04:00 | 347.23 | 348.75 | 331.7  |  333.87 | 8.11457e+07 |           0 |              0 |
| 2025-09-02 00:00:00-04:00 | 328.23 | 333.33 | 325.6  |  329.36 | 5.8392e+07  |           0 |              0 |
| 2025-09-03 00:00:00-04:00 | 335.2  | 343.33 | 328.51 |  334.09 | 8.87333e+07 |           0 |              0 |
| 2025-09-04 00:00:00-04:00 | 336.15 | 338.89 | 331.48 |  338.53 | 6.0711e+07  |           0 |              0 |
| 2025-09-05 00:00:00-04:00 | 348    | 355.87 | 344.68 |  350.84 | 1.08772e+08 |           0 |              0 |


'| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-08-29 00:00:00-04:00 | 347.23 | 348.75 | 331.7  |  333.87 | 8.11457e+07 |           0 |              0 |\n| 2025-09-02 00:00:00-04:00 | 328.23 | 333.33 | 325.6  |  329.36 | 5.8392e+07  |           0 |              0 |\n| 2025-09-03 00:00:00-04:00 | 335.2  | 343.33 | 328.51 |  334.09 | 8.87333e+07 |           0 |              0 |\n| 2025-09-04 00:00:00-04:00 | 336.15 | 338.89 | 331.48 |  338.53 | 6.0711e+07  |           0 |              0 |\n| 2025-09-05 00:00:00-04:00 | 348    | 355.87 | 344.68 |  350.84 | 1.08772e+08 |           0 |              0 |'

In [28]:
get_yf_stock_recommendations('PLTR')

|    | period   |   strongBuy |   buy |   hold |   sell |   strongSell |
|---:|:---------|------------:|------:|-------:|-------:|-------------:|
|  0 | 0m       |           1 |     3 |     17 |      2 |            2 |
|  1 | -1m      |           1 |     3 |     17 |      2 |            2 |
|  2 | -2m      |           1 |     3 |     16 |      3 |            2 |
|  3 | -3m      |           1 |     3 |     15 |      4 |            2 |


'|    | period   |   strongBuy |   buy |   hold |   sell |   strongSell |\n|---:|:---------|------------:|------:|-------:|-------:|-------------:|\n|  0 | 0m       |           1 |     3 |     17 |      2 |            2 |\n|  1 | -1m      |           1 |     3 |     17 |      2 |            2 |\n|  2 | -2m      |           1 |     3 |     16 |      3 |            2 |\n|  3 | -3m      |           1 |     3 |     15 |      4 |            2 |'

In [32]:
# 질문 테스트: 엔비디아 주가가 최근 하락한 이유가 뭐야?

client = OpenAI()

def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages = messages,
        tools=tools,
    )
    return response

messages = [
    {"role":"system", "content":"너는 사용자를 도와주는 친절한 조력자야."}
]

while True:
    user_input = input("사용자: ")
    if user_input == "exit":
        break
    messages.append({"role": "user", "content": user_input})
    response = get_ai_response(messages, tools=tools)
    ai_msg = response.choices[0].message
    print(ai_msg)

    tool_calls = ai_msg.tool_calls

    # 하나의 지역 시간을 묻는 경우
    if tool_calls:
        tool_name = tool_calls[0].function.name
        tool_call_id = tool_calls[0].id

        arguments = json.loads(tool_calls[0].function.arguments) # str -> dict

        if tool_name == "get_current_time":
            messages.append({
                "role": "function",
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_current_time(timezone=arguments['timezone']),
            })
        response = get_ai_response(messages, tools=tools)
        ai_msg = response.choices[0].message

    tool_calls = ai_msg.tool_calls 
    if tool_calls: 
        for tool_call in tool_calls:
            tool_name = tool_call.function.name 
            tool_call_id = tool_call.id   
            arguments = json.loads(tool_call.function.arguments) 
            
            if tool_name == "get_current_time":
                func_result = get_current_time(timezone=arguments['timezone']), 

            elif tool_name == "get_yf_stock_info":
                func_result = get_yf_stock_info(ticker=arguments['ticker'])

            elif tool_name == "get_yf_stock_history":
                func_result = get_yf_stock_history(ticker=arguments['ticker'], period=arguments['period'])   

            elif tool_name == "get_yf_stock_recommendations":
                func_result = get_yf_stock_recommendations(ticker=arguments['ticker'])     

            messages.append({
                "role": "function",
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": func_result,
            })                                        
        
        messages.append({"role":"system", "content":"이제 주어진 결과를 참고해서 답변해."})
        response = get_ai_response(messages, tools=tools)
        ai_msg = response.choices[0].message

    messages.append(ai_msg)
    print('\n===== AI =====')
    print(ai_msg.content)
    print('============\n')

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_G59zYvQexFPjxkSM3Z3C6VtT', function=Function(arguments='{"ticker":"NVDA"}', name='get_yf_stock_info'), type='function')])
{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation, a computing infrastructure company, provides graphics and compute and networking solutions in the United States, Singapore, Taiwan, China, Hong Kong, and internationally. The Compute & Networking segment includes its Data Centre accelerated computing platforms and artificial intelligence solution